In [1]:
with open("C:/Users/bhudi/Desktop/New folder/git/txtgen/DL.txt",'r',encoding='utf-8') as file:
    txt=file.read()

In [2]:
print(txt)

Deep learning is a subset of machine learning that is inspired by the structure and function of the human brain. It involves the use of artificial neural networks, which are computational models that mimic the way neurons in the brain process information. Deep learning algorithms are designed to learn from large amounts of data, extracting patterns and features that can be used for a variety of tasks, such as image recognition, natural language processing, and decision-making.
At the core of deep learning are neural networks, which are composed of interconnected nodes, called neurons, that are organized into layers. These layers can be broadly categorized into three types: input layers, hidden layers, and output layers. The input layer receives the raw data, such as an image or text, and passes it to the hidden layers. The hidden layers perform transformations on the data, extracting features and patterns that are relevant to the task at hand. The output layer produces the final result

In [3]:
from keras.preprocessing.text import Tokenizer

In [4]:
tk=Tokenizer()

In [5]:
tk.fit_on_texts([txt])

In [6]:
tk.word_index

{'and': 1,
 'the': 2,
 'learning': 3,
 'deep': 4,
 'to': 5,
 'of': 6,
 'in': 7,
 'is': 8,
 'as': 9,
 'a': 10,
 'for': 11,
 'that': 12,
 'are': 13,
 'models': 14,
 'data': 15,
 'networks': 16,
 'can': 17,
 'such': 18,
 'neural': 19,
 'more': 20,
 'with': 21,
 'from': 22,
 'be': 23,
 'these': 24,
 'on': 25,
 'has': 26,
 'layers': 27,
 'been': 28,
 'it': 29,
 'algorithms': 30,
 'learn': 31,
 'used': 32,
 'image': 33,
 'or': 34,
 'its': 35,
 'need': 36,
 'there': 37,
 'speech': 38,
 'have': 39,
 'which': 40,
 'features': 41,
 'tasks': 42,
 'natural': 43,
 'language': 44,
 'processing': 45,
 'one': 46,
 'domains': 47,
 'applications': 48,
 'systems': 49,
 'challenges': 50,
 'research': 51,
 'new': 52,
 'machine': 53,
 'recognition': 54,
 'making': 55,
 'output': 56,
 'task': 57,
 'time': 58,
 'various': 59,
 'this': 60,
 'training': 61,
 'many': 62,
 'generation': 63,
 'also': 64,
 'challenge': 65,
 'another': 66,
 'efficient': 67,
 'other': 68,
 'by': 69,
 'human': 70,
 'brain': 71,
 'comp

In [7]:
tot_words=len(tk.word_index)+1
print(tot_words)

447


In [8]:
ip=[]
for i in txt.split('\n'):
    tokens=tk.texts_to_sequences([i])[0]
    for j in range(1,len(tokens)):
        ngramseq=tokens[:j+1]
        ip.append(ngramseq)
        # print(ip)

In [9]:
maxseqlen=max([len(seq) for seq in ip])

In [10]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [11]:
ip_seq=np.array(pad_sequences(ip,maxlen=maxseqlen,padding='pre'))

In [12]:
ip_seq[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 3])

In [13]:
X=ip_seq[:,:-1]
y=ip_seq[:,-1]

In [14]:
y

array([  3,   8,  10, ...,   1, 173, 174])

In [15]:
from keras.utils import to_categorical

In [16]:
y=np.array(to_categorical(y,num_classes=tot_words))

In [17]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM

In [19]:
from keras.callbacks import EarlyStopping

In [20]:
es=EarlyStopping(monitor='loss',patience=5,verbose=2)

In [21]:
mod=Sequential()
mod.add(Embedding(tot_words,100,input_length=maxseqlen-1))
mod.add(LSTM(150))
mod.add(Dense(tot_words,activation='softmax'))
mod.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
mod.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 98, 100)           44700     
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 447)               67497     
                                                                 
Total params: 262797 (1.00 MB)
Trainable params: 262797 (1.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
mod.fit(X,y,batch_size=64,epochs=100,verbose=2,callbacks=[es])

Epoch 1/100
20/20 - 4s - loss: 6.0079 - accuracy: 0.0331 - 4s/epoch - 206ms/step
Epoch 2/100
20/20 - 2s - loss: 5.5438 - accuracy: 0.0541 - 2s/epoch - 81ms/step
Epoch 3/100
20/20 - 2s - loss: 5.4162 - accuracy: 0.0590 - 2s/epoch - 79ms/step
Epoch 4/100
20/20 - 2s - loss: 5.3857 - accuracy: 0.0590 - 2s/epoch - 78ms/step
Epoch 5/100
20/20 - 2s - loss: 5.3635 - accuracy: 0.0590 - 2s/epoch - 81ms/step
Epoch 6/100
20/20 - 2s - loss: 5.3291 - accuracy: 0.0816 - 2s/epoch - 81ms/step
Epoch 7/100
20/20 - 2s - loss: 5.2708 - accuracy: 0.0856 - 2s/epoch - 86ms/step
Epoch 8/100
20/20 - 2s - loss: 5.1903 - accuracy: 0.0921 - 2s/epoch - 78ms/step
Epoch 9/100
20/20 - 2s - loss: 5.0953 - accuracy: 0.0864 - 2s/epoch - 79ms/step
Epoch 10/100
20/20 - 2s - loss: 4.9826 - accuracy: 0.0994 - 2s/epoch - 98ms/step
Epoch 11/100
20/20 - 2s - loss: 4.8577 - accuracy: 0.1163 - 2s/epoch - 102ms/step
Epoch 12/100
20/20 - 2s - loss: 4.7244 - accuracy: 0.1430 - 2s/epoch - 88ms/step
Epoch 13/100
20/20 - 2s - loss: 4.5

In [23]:
inputtxt="CNNs"
words=30

for _ in range(words):
    tokens=tk.texts_to_sequences([inputtxt])[0]
    print(tokens)
    tokens=pad_sequences([tokens], maxlen=maxseqlen-1, padding='pre')
    pred=np.argmax(mod.predict(tokens),axis=-1)
    op=""
    for wrd,idx in tk.word_index.items():
        if idx==pred:
            op=wrd
            break
    inputtxt+=" "+wrd

print(inputtxt)

[132]
1/1 [==============================] - 0s 477ms/step
[132, 13]
1/1 [==============================] - 0s 21ms/step
[132, 13, 230]
1/1 [==============================] - 0s 14ms/step
[132, 13, 230, 117]
1/1 [==============================] - 0s 23ms/step
[132, 13, 230, 117, 6]
1/1 [==============================] - 0s 29ms/step
[132, 13, 230, 117, 6, 19]
1/1 [==============================] - 0s 26ms/step
[132, 13, 230, 117, 6, 19, 16]
1/1 [==============================] - 0s 23ms/step
[132, 13, 230, 117, 6, 19, 16, 32]
1/1 [==============================] - 0s 11ms/step
[132, 13, 230, 117, 6, 19, 16, 32, 7]
1/1 [==============================] - 0s 36ms/step
[132, 13, 230, 117, 6, 19, 16, 32, 7, 4]
1/1 [==============================] - 0s 24ms/step
[132, 13, 230, 117, 6, 19, 16, 32, 7, 4, 3]
1/1 [==============================] - 0s 25ms/step
[132, 13, 230, 117, 6, 19, 16, 32, 7, 4, 3, 231]
1/1 [==============================] - 0s 22ms/step
[132, 13, 230, 117, 6, 19, 16, 32, 7

In [24]:
mod.save('100e.h5')

C:\Users\bhudi\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
import pickle

In [27]:
with open('tok.pkl', 'wb') as handle:
    pickle.dump(tk, handle, protocol=pickle.HIGHEST_PROTOCOL)